In [1]:
import functools

import numpy as np
import tensorflow as tf

# CSV Data

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file('train.csv', TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval.csv', TEST_DATA_URL)

24576/13049 [========================================================] - 0s 0us/step


In [3]:
np.set_printoptions(precision=3, suppress=True)

In [4]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [5]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs
    )

    return dataset    

In [6]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [7]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [8]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [28.  40.5 25.  65.   2. ]
n_siblings_spouses  : [0 0 1 0 1]
parch               : [0 2 0 0 1]
fare                : [ 7.05  14.5    7.925 26.55  26.   ]
class               : [b'Third' b'Third' b'Third' b'First' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'n']


In [10]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'male' b'female' b'female']
age                 : [28. 52. 33. 41. 52.]
n_siblings_spouses  : [1 0 0 0 1]
parch               : [0 0 0 1 1]
fare                : [51.862 30.5    8.654 19.5   93.5  ]
class               : [b'First' b'First' b'Third' b'Second' b'First']
deck                : [b'D' b'C' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'n']


In [12]:
# 일부 열(column)만 선택할 수 있다.

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [22. 28. 40. 40. 28.]
n_siblings_spouses  : [0 1 0 1 0]
class               : [b'Third' b'First' b'First' b'First' b'Third']
deck                : [b'unknown' b'C' b'C' b'E' b'unknown']
alone               : [b'y' b'n' b'y' b'n' b'y']


# Pre-processing

In [13]:
# 이미 숫자 형식인 경우 벡터로 묶을 수 있다.

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']

DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]

temp_dataset = get_dataset(train_file_path,
                           select_columns=SELECT_COLUMNS,
                           column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                 : [19.  4. 41. 16. 36.]
n_siblings_spouses  : [0. 1. 0. 0. 0.]
parch               : [0. 1. 1. 0. 0.]
fare                : [ 6.75  23.    19.5   26.     7.496]


In [14]:
example_batch, labels_batch = next(iter(temp_dataset))

In [15]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [16]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[  4.      1.      1.     23.   ]
 [ 28.      0.      0.     31.   ]
 [ 28.      0.      0.      7.796]
 [ 50.      0.      1.    247.521]
 [ 28.      0.      0.      8.458]]

[1 0 0 1 0]


### 혼합 데이터 유형이 있는 경우, 해당 단순 숫자 필드를 분리할 수 있다. ```tf.feature_column``` API로 처리 가능하지만 약간의 오버헤드가 발생하므로 사용하지 않는 것이 좋다.

In [17]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'female']
age                 : [34.  36.  44.  55.5 28. ]
n_siblings_spouses  : [0 1 0 0 3]
parch               : [1 0 0 0 1]
fare                : [23.    78.85  27.721  8.05  25.467]
class               : [b'Second' b'First' b'First' b'Third' b'Third']
deck                : [b'unknown' b'C' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'n']


In [18]:
example_batch, labels_batch = next(iter(temp_dataset))

### 숫자 특성 목록을 선택하고 단일 column으로 묶는 것보다 일반적인 전처리기를 정의.

In [19]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [20]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [21]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'D' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'y']
numeric             : [[32.     0.     0.     7.854]
 [16.     0.     1.    39.4  ]
 [30.     1.     1.    24.15 ]
 [21.     0.     0.     8.663]
 [39.     0.     0.    13.   ]]


In [22]:
example_batch, labels_batch = next(iter(packed_train_data))

# 데이터 정규화
- 연속 데이터는 항상 정규화 되어야 한다.

In [23]:
import pandas as pd

In [24]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [27]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [28]:
def normalize_numeric_data(data, mean, std):
    return (data-mean) / std

In [29]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f63977553b0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [30]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 32.   ,   0.   ,   0.   ,   7.925],
       [ 28.   ,   0.   ,   0.   ,   7.896],
       [ 28.   ,   1.   ,   0.   , 146.521],
       [ 18.   ,   0.   ,   0.   ,   9.842],
       [ 38.   ,   0.   ,   0.   ,  80.   ]], dtype=float32)>

In [32]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 0.189, -0.474, -0.479, -0.485],
       [-0.13 , -0.474, -0.479, -0.485],
       [-0.13 ,  0.395, -0.479,  2.054],
       [-0.93 , -0.474, -0.479, -0.45 ],
       [ 0.669, -0.474, -0.479,  0.835]], dtype=float32)

In [33]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [34]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [35]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [36]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


### ```tf.keras.layers.DenseFeatures```에 전달하여 두 input 유형을 추출하고 전처리할 입력 레이어를 만든다.

In [37]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

In [38]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.189 -0.474
 -0.479 -0.485  1.     0.   ]


# Model Build
- ```preprocessing_layer```를 사용하여 ```tf.keras.Sequential```을 빌드

In [39]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Training

In [40]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [41]:
train_data

<ShuffleDataset element_spec=(OrderedDict([('sex', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('class', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('deck', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('embark_town', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('alone', TensorSpec(shape=(None,), dtype=tf.string, name=None)), ('numeric', TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))]), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [42]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 6s 9ms/step - loss: 0.5145 - accuracy: 0.7321
Epoch 2/20
126/126 [==============================] - 1s 9ms/step - loss: 0.4257 - accuracy: 0.8214
Epoch 3/20
126/126 [==============================] - 1s 8ms/step - loss: 0.4038 - accuracy: 0.8357
Epoch 4/20
126/126 [==============================] - 1s 7ms/step - loss: 0.3957 - accuracy: 0.8309
Epoch 5/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3793 - accuracy: 0.8293
Epoch 6/20
126/126 [==============================] - 1s 8ms/step - loss: 0.3719 - accuracy: 0.8453
Epoch 7/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3671 - accuracy: 0.8501
Epoch 8/20
126/126 [==============================] - 1s 8ms/step - loss: 0.3578 - accuracy: 0.8581
Epoch 9/20
126/126 [==============================] - 1s 8ms/step - loss: 0.3503 - accuracy: 0.8453
Epoch 10/20
126/126 [==============================] - 1s 8ms/step - loss: 0.3467 - accuracy: 0.8612

# Test

In [47]:
test_loss, test_accuracy = model.evaluate(test_data)

print("\n\nTest Loss {}, Test Accuracy {}".format(test_loss, test_accuracy))

53/53 [==============================] - 1s 7ms/step - loss: 0.4461 - accuracy: 0.8409


Test Loss 0.4460510015487671, Test Accuracy 0.8409090638160706


In [50]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 82.10%  | Actual outcome:  SURVIVED
Predicted survival: 26.45%  | Actual outcome:  DIED
Predicted survival: 99.34%  | Actual outcome:  SURVIVED
Predicted survival: 2.85%  | Actual outcome:  DIED
Predicted survival: 11.71%  | Actual outcome:  DIED
